In [1]:
import sys
sys.path.append('/home/jupyter-wangxm/6_clickhouse_raw_data/script')

In [2]:
import s1_feature_test as s1
import pandas as pd

'input current date: '

20191016


## 字段名称：是否员工

字段业务逻辑：


'''



'''

## 开始测试

### login clickhouse

In [3]:
PD = s1.Prepare_Data()
PD.login_and_choose_database(database = 'test_database') # choose database；
PD.raw_query('show tables;') # check tables exist；

[]

### 创建中间表

create table

In [7]:
PD.create_table(table_name = '_1016_STF_EMP_IND_1', # 以’字段名称 + 中间表序号‘进行中间表命名，
             columns = 'cust_id String, date Date',
             engine = 'MergeTree',
             ordered_column = 'cust_id')

query overview: 
CREATE TABLE IF NOT EXISTS _1016_STF_EMP_IND_1
 (cust_id String, date Date)
 ENGINE = MergeTree
 ORDER BY cust_id

show tables: 
[('_1016_STF_EMP_IND_1',)]


sample data

In [8]:
PD.get_random_sample(current_table = 'python_clickhouse_test.basic_table',
                    new_table = '_1016_STF_EMP_IND_2',
                    num_cases = '10',
                    ordered_column = 'cust_id',
                    selected_columns = '*')

query overview: 
CREATE TABLE IF NOT EXISTS _1016_STF_EMP_IND_2
 ENGINE = MergeTree
 ORDER BY cust_id
 AS SELECT *
 FROM python_clickhouse_test.basic_table
 ORDER BY rand() 
 LIMIT 10

show tables: 
[('_1016_STF_EMP_IND_1',), ('_1016_STF_EMP_IND_2',)]


merge data

In [10]:
PD.merge_data(new_table = '_1016_STF_EMP_IND_3',
             ordered_column = 'cust_id',
             joined_method = 'inner join',
             left_table_query = ' select cust_id as cust_id_basic, date as date_basic from python_clickhouse_test.basic_table',
             right_table_query = ' select * from python_clickhouse_test.information_table',
             on_query = 'a.cust_id_basic = b.cust_id \n AND a.date_basic = b.date'
             )

query overview: 
CREATE TABLE IF NOT EXISTS _1016_STF_EMP_IND_3
ENGINE = MergeTree
ORDER BY cust_id
AS SELECT * FROM ( 
 select cust_id as cust_id_basic, date as date_basic from python_clickhouse_test.basic_table)a 
inner join( 
 select * from python_clickhouse_test.information_table)b 
ON a.cust_id_basic = b.cust_id 
 AND a.date_basic = b.date

show tables: 
[('_1016_STF_EMP_IND_1',), ('_1016_STF_EMP_IND_2',), ('_1016_STF_EMP_IND_3',)]


other sql

In [11]:
PD.raw_query('desc _1016_STF_EMP_IND_3;')





[('cust_id_basic', 'String', '', '', '', ''),
 ('date_basic', 'Date', '', '', '', ''),
 ('cust_id', 'String', '', '', '', ''),
 ('date', 'Date', '', '', '', ''),
 ('gender', 'String', '', '', '', ''),
 ('job', 'String', '', '', '', '')]

## 结果比对

In [12]:
res_kx, res_jh = PD.check_res(table_kx = '_1016_STF_EMP_IND_3',
                              table_jh = 'python_clickhouse_test.information_table',
                              columns_kx = 'cust_id_basic, date_basic, job',
                              columns_jh = 'cust_id, date, job',
                              columns_comp = 'job',
                              header = ['cust_id', 'date', 'job'])

query overview: 
['SELECT cust_id_basic, date_basic, job FROM _1016_STF_EMP_IND_3'],
['SELECT cust_id, date, job FROM python_clickhouse_test.information_table']

check merged shape: res_kx_df: 500000, res_jh_df: 500000, res_comp: 500000

There are 0 unequal cases. (check detail if >0)

A glance of merged_data:


,cust_id,date,job_kx,job_jh
0,2979798304,2018-08-21,Accounter,Accounter
1,2979835241,2018-09-08,Artist,Artist
2,2979853619,2018-09-23,Professional Manager,Professional Manager
3,2979855230,2018-08-26,~,~
4,2979855476,2018-11-25,Accounter,Accounter
